In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter

from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from catboost import Pool, CatBoostClassifier
import lightgbm as lgb


In [3]:
#Importing
train_data = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/training_v2.csv')
test_data =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/unlabeled.csv')

sub_data = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/samplesubmission.csv')
data_dict = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/WiDS Datathon 2020 Dictionary.csv')

In [4]:
def weighted_classt(x): 
    if pd.isna(x):
        return np.nan
    elif x < 15: 
        return 'very severely underweight' 
    elif x >= 15 and x < 16: 
        return 'severely weight' 
    elif x >=16 and x < 18.5: 
        return 'underweight' 
    elif x >= 18.5 and x < 25: 
        return 'healthy weight' 
    elif x >= 25 and x < 30: 
        return 'overweight'
    elif x >= 30 and x < 35: 
        return 'class 1' 
    elif x >= 35 and x < 40: 
        return 'class 2' 
    else: 
        return 'class 3' 

In [5]:
train_data['weightclass'] = train_data['bmi'].map(weighted_classt)
test_data['weightclass'] = test_data['bmi'].map(weighted_classt)

In [6]:
cat_var = list(data_dict[data_dict['Data Type'].isin(['binary','string'])]['Variable Name'])
cat_var.remove('icu_admit_type')
cat_var.remove('bmi')
cat_var = cat_var[1:]

In [7]:
#cat_var = cat_var[1:]
#cat_var = cat_var[1:]
cat_var = cat_var + ['weightclass']

In [8]:
cont_var = list(data_dict[data_dict['Data Type'].isin(['integer','numeric'])]['Variable Name'])
cont_var.remove('pred')
cont_var = cont_var + ['bmi']

#### Baseline

In [9]:
x_train = train_data.drop('hospital_death',axis=1)
y_train = train_data['hospital_death']

In [10]:
x_test = test_data.drop('hospital_death',axis=1)

In [11]:
x_comb = pd.concat([x_train,x_test],sort=False)

In [12]:
### impute bmi values based on height and weight
count = 0
bmi_list = []
for i in tqdm(range(x_comb.shape[0])):
    
    if pd.isnull(x_comb["weight"].iloc[i]) or pd.isnull(x_comb["height"].iloc[i]):
        bmi_list.append(x_comb["bmi"].iloc[i])
        continue
    else:
        
        count += 1
        bmi_list.append(x_comb["weight"].iloc[i] / ((x_comb["height"].iloc[i]/100) ** 2))

100%|██████████| 131021/131021 [00:10<00:00, 12016.41it/s]


In [13]:
x_comb['bmi'] = bmi_list

In [14]:
new_cat_var = list(set(cat_var).intersection(set(x_comb.columns)))

In [15]:
for col in tqdm(new_cat_var):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col].astype(str))

100%|██████████| 26/26 [00:03<00:00,  7.21it/s]


In [16]:
new_cont_var = list(set(cont_var).intersection(set(x_comb.columns)))

In [ ]:
# for col in tqdm(new_cont_var):
#     x_comb[col] = x_comb[col].fillna(x_comb[col].mean())

In [17]:
count_var = []
for i,col in tqdm(enumerate(new_cat_var)):
    counter = Counter(x_comb[col])
    x_comb[str(col)+'count'] = x_comb[col].apply(lambda x:counter[x])
    count_var.append(str(col)+'count')

26it [00:02,  9.06it/s]


In [18]:
age_cat = []

for i in tqdm(range(0,len(x_comb))):
    
    val = x_comb['age'].iloc[i]
    
    if val >= 15 and val <= 24: 
        age_cat.append('igen')
    
    elif val >= 25 and val <= 54: 
       age_cat.append('Prime_working_Age')
    
    elif val >= 55 and val <= 64: 
        age_cat.append('Mature_working_Age')
        
    else: 
        age_cat.append('Elderly_working_Age')


100%|██████████| 131021/131021 [00:03<00:00, 40550.03it/s]


In [19]:
x_comb['age_category'] = age_cat
le = LabelEncoder()
x_comb['age_category'] = le.fit_transform(x_comb['age_category'].astype(str))

In [20]:
list_illness = ['aids',
                'cirrhosis',
                'diabetes_mellitus',
                'hepatic_failure',
                'immunosuppression',
                'leukemia',
                'lymphoma',
                'solid_tumor_with_metastasis']

In [21]:
inter_cols1 = []

for col in tqdm(list_illness) :
    x_comb['hospital_id'+str(col)] = x_comb['hospital_id'].astype(str) + '_' + x_comb[col].astype(str)
    x_comb['gender'+str(col)] = x_comb['gender'].astype(str) + '_' + x_comb[col].astype(str)
    x_comb['age'+str(col)] = x_comb['age'].astype(str) + '_' + x_comb[col].astype(str)
    x_comb['bmi'+str(col)] = x_comb['bmi'].astype(str) + '_' + x_comb[col].astype(str)
    
    inter_cols1.append('hospital_id'+str(col))
    inter_cols1.append('gender'+str(col))
    inter_cols1.append('age'+str(col))
    inter_cols1.append('bmi'+str(col))

100%|██████████| 8/8 [00:09<00:00,  1.22s/it]


In [22]:
for cols in tqdm(inter_cols1):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols])
    

100%|██████████| 32/32 [00:02<00:00, 11.89it/s]


In [23]:
inter_cols2 = []

for col in tqdm(list_illness) :
    x_comb['age_category'+str(col)] = x_comb['age_category'].astype(str) + '_' + x_comb[col].astype(str)
    inter_cols2.append('age_category'+str(col))

100%|██████████| 8/8 [00:02<00:00,  2.88it/s]


In [24]:
for cols in tqdm(inter_cols2):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols])
    

100%|██████████| 8/8 [00:00<00:00, 19.76it/s]


In [25]:
inter_cols3 = []

for col1 in tqdm(list_illness) :
    for col2 in list_illness:
        if col1 !=col2 :
            x_comb[col1+col2] = x_comb[col1].astype(str) + '_' + x_comb[col2].astype(str)
            inter_cols3.append(col1+col2)

100%|██████████| 8/8 [00:16<00:00,  2.05s/it]


In [26]:
for cols in tqdm(inter_cols3):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols])
    

100%|██████████| 56/56 [00:01<00:00, 36.30it/s]


In [27]:
#train_data.columns[0:100]

list_imp_cols = ['icu_type','bmi','age','gender','d1_heartrate_max']

inter_cols4 = []

for col1 in tqdm(list_imp_cols) :
    for col2 in list_imp_cols:
        if col1 !=col2 :
            x_comb[col1+col2] = x_comb[col1].astype(str) + '_' + x_comb[col2].astype(str)
            inter_cols4.append(col1+col2)

100%|██████████| 5/5 [00:03<00:00,  1.45it/s]


In [28]:
for cols in tqdm(inter_cols4):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols])

100%|██████████| 20/20 [00:02<00:00,  8.30it/s]


In [29]:
final_cat_var = new_cat_var + count_var + ['age_category'] + inter_cols1 + inter_cols2 + inter_cols3 + inter_cols4

##### Target Encoding

In [30]:
temp_train = x_train.copy()
temp_train['hospital_death'] = train_data['hospital_death']

In [31]:
cat_count_var = []

for i,col in tqdm(enumerate(new_cat_var)):
    mean = temp_train['hospital_death'].mean()
    
    #Compute the number of values and the mean of each group
    agg = temp_train.groupby(col)['hospital_death'].agg(['count', 'mean'])
    counts = agg['count']
    means =  agg['mean']
    
    #Compute the "smoothed" means
    m=3
    smooth = (counts * means + m * mean) / (counts + m)
    
    #Final_val
    x_comb[str(col)+'count_new'] = x_comb[col].map(smooth)
    cat_count_var.append(str(col)+'count_new')
    

26it [00:02, 12.84it/s]


In [32]:
# # Remove Features with more than 70 percent missing values
data_missing = (x_comb.isnull().sum() / len(x_comb)).sort_values(ascending = False)
data_missing = data_missing.index[data_missing > 0.93]

x_comb = x_comb.drop(columns = data_missing)

In [33]:
cat_count_var_new = list(set(cat_count_var).intersection(x_comb.columns))

In [34]:
for col in tqdm(cat_count_var_new):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col].astype(str))

100%|██████████| 15/15 [00:07<00:00,  2.04it/s]


In [38]:
aggs = {}

for col in tqdm(count_var):
    aggs[col] = ['nunique']
    

100%|██████████| 26/26 [00:00<00:00, 45287.34it/s]


In [39]:
for col in tqdm(count_var):
    
    aggs_temp = aggs.copy()
    aggs_temp.pop(col)
    
    agg_df = x_comb.groupby(col).agg(aggs_temp).reset_index()
    agg_df.columns = [col] + [col + '_' + c[0] +'_' + c[1] for c in agg_df.columns[1:]]
    
    x_comb = pd.merge(left = x_comb, right= agg_df, how='left',
                    left_on=[col], right_on=[col])
    

100%|██████████| 26/26 [02:59<00:00,  6.92s/it]


In [40]:
x_comb = x_comb.drop(['encounter_id','patient_id'],axis=1)

In [41]:
train_x = x_comb[0:91713]
test_x = x_comb[91713:]

In [43]:
x_comb.shape

(131021, 992)

In [42]:
x1_train, x1_val, y1_train, y1_val = train_test_split(train_x, y_train, 
                                                      test_size=0.2,
                                                      random_state=0,
                                                      stratify = y_train)

In [44]:
random_state = 42
np.random.seed(random_state)

lgb_params = {
    'boost': 'gbdt',
    'learning_rate': 0.01,
    'max_depth':8,
    'metric':{'auc'},
    'num_threads': -1,
    'objective': 'binary',
    'verbosity': 1,
    'num_leaves':16,
    'bagging_fraction':0.5, #rows
    'bagging_frequency':5,
    'min_data_in_leaf':50,
    'feature_fraction':0.3, # features
    'lambda_l1':10,
    'lambda_l2':15,
    'bagging_seed':2019,
    'min_gain_split':0.3
}

In [45]:
trn_data = lgb.Dataset(x1_train, label= y1_train)
val_data = lgb.Dataset(x1_val,  label=  y1_val)

In [46]:
lgb_clf = lgb.train(lgb_params,
                    trn_data,
                    13000,
                    valid_sets = [trn_data, val_data],
                    early_stopping_rounds=3000,
                    verbose_eval=200
                    )

Training until validation scores don't improve for 3000 rounds.
[200]	training's auc: 0.896157	valid_1's auc: 0.884783
[400]	training's auc: 0.905967	valid_1's auc: 0.891729
[600]	training's auc: 0.912826	valid_1's auc: 0.896235
[800]	training's auc: 0.91766	valid_1's auc: 0.898817
[1000]	training's auc: 0.921626	valid_1's auc: 0.900568
[1200]	training's auc: 0.925089	valid_1's auc: 0.901852
[1400]	training's auc: 0.928243	valid_1's auc: 0.902786
[1600]	training's auc: 0.931031	valid_1's auc: 0.903512
[1800]	training's auc: 0.933448	valid_1's auc: 0.904094
[2000]	training's auc: 0.93567	valid_1's auc: 0.904548
[2200]	training's auc: 0.937659	valid_1's auc: 0.904831
[2400]	training's auc: 0.93963	valid_1's auc: 0.905132
[2600]	training's auc: 0.94139	valid_1's auc: 0.905349
[2800]	training's auc: 0.943053	valid_1's auc: 0.905587
[3000]	training's auc: 0.944694	valid_1's auc: 0.905816
[3200]	training's auc: 0.946286	valid_1's auc: 0.905932
[3400]	training's auc: 0.947869	valid_1's auc: 0

In [ ]:
pred = lgb_clf.predict(test_x)

In [ ]:
data_sub = x_test[['encounter_id']]
data_sub['hospital_death'] = pred

In [ ]:
data_sub.to_csv('/Users/s0c02nj/Desktop/WiDS/sub2_lgb_01022020.csv',index = False)

In [47]:
kf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

In [48]:
predictions = np.zeros(len(x_test))
num_models = 0

for train_index, valid_index in tqdm(kf.split(train_x, y_train)):
    
    d_train = lgb.Dataset(train_x.iloc[train_index], label=y_train[train_index])
    d_val = lgb.Dataset(train_x.iloc[valid_index], label=y_train[valid_index])

    clf = lgb.train(lgb_params, d_train, 130000, verbose_eval=1000, 
                    valid_sets = [d_train, d_val], early_stopping_rounds = 3000)
    

    
    num_models += 1
    
    predictions += clf.predict(test_x)

0it [00:00, ?it/s]

Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.920117	valid_1's auc: 0.902579
[2000]	training's auc: 0.933235	valid_1's auc: 0.9062
[3000]	training's auc: 0.942008	valid_1's auc: 0.907603
[4000]	training's auc: 0.949123	valid_1's auc: 0.907968
[5000]	training's auc: 0.955403	valid_1's auc: 0.908222
[6000]	training's auc: 0.961027	valid_1's auc: 0.908363
[7000]	training's auc: 0.965895	valid_1's auc: 0.908313
[8000]	training's auc: 0.970086	valid_1's auc: 0.908272
[9000]	training's auc: 0.973905	valid_1's auc: 0.908333
Early stopping, best iteration is:
[6689]	training's auc: 0.964456	valid_1's auc: 0.908451


1it [03:19, 199.42s/it]

Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.91915	valid_1's auc: 0.907779
[2000]	training's auc: 0.93256	valid_1's auc: 0.912249
[3000]	training's auc: 0.941381	valid_1's auc: 0.913636
[4000]	training's auc: 0.948432	valid_1's auc: 0.914318
[5000]	training's auc: 0.954674	valid_1's auc: 0.914627
[6000]	training's auc: 0.960264	valid_1's auc: 0.914629
[7000]	training's auc: 0.965177	valid_1's auc: 0.914662
[8000]	training's auc: 0.96958	valid_1's auc: 0.914632
[9000]	training's auc: 0.973376	valid_1's auc: 0.914619
[10000]	training's auc: 0.976803	valid_1's auc: 0.914397
Early stopping, best iteration is:
[7470]	training's auc: 0.967341	valid_1's auc: 0.914704


2it [07:08, 208.38s/it]

...

8it [28:35, 204.81s/it]

Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.919853	valid_1's auc: 0.903987
[2000]	training's auc: 0.933147	valid_1's auc: 0.907318
[3000]	training's auc: 0.94201	valid_1's auc: 0.908192
[4000]	training's auc: 0.949046	valid_1's auc: 0.908357
[5000]	training's auc: 0.955068	valid_1's auc: 0.908437
[6000]	training's auc: 0.960661	valid_1's auc: 0.908455
[7000]	training's auc: 0.965638	valid_1's auc: 0.908523
[8000]	training's auc: 0.969926	valid_1's auc: 0.90834
[9000]	training's auc: 0.973737	valid_1's auc: 0.908257
[10000]	training's auc: 0.977115	valid_1's auc: 0.907904
Early stopping, best iteration is:
[7124]	training's auc: 0.96619	valid_1's auc: 0.908576


9it [31:53, 202.70s/it]

Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.919515	valid_1's auc: 0.906062
[2000]	training's auc: 0.932855	valid_1's auc: 0.910475
[3000]	training's auc: 0.941545	valid_1's auc: 0.911891
[4000]	training's auc: 0.948608	valid_1's auc: 0.912366
[5000]	training's auc: 0.954897	valid_1's auc: 0.912562
[6000]	training's auc: 0.960501	valid_1's auc: 0.912759
[7000]	training's auc: 0.965461	valid_1's auc: 0.912784
[8000]	training's auc: 0.969787	valid_1's auc: 0.912835
[9000]	training's auc: 0.973611	valid_1's auc: 0.912665
[10000]	training's auc: 0.977027	valid_1's auc: 0.912705
Early stopping, best iteration is:
[7393]	training's auc: 0.967148	valid_1's auc: 0.912876


10it [35:13, 211.38s/it]


In [49]:
predictions_10 = predictions / 10.0

In [50]:
data_sub = x_test[['encounter_id']]

In [51]:
data_sub['hospital_death'] = predictions_10

/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [52]:
data_sub.to_csv('/Users/s0c02nj/Desktop/WiDS/sub_lgb_keeping_missing_as_it_is_no_cat_agg_10th_feb.csv',index = False)

In [ ]:
a1 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/Submission/sub_ensemble.csv')
a2 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/Submission/sub_ensemble_gm.csv')
a3 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/Submission/sub_ensemble_gm1.csv')
a4 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_ensemble_gm2.csv')
a5 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub8_lgb_10fold1.csv')
a6 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub8_lgb_10fold.csv')
a7 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub10_lgb_kfold2.csv')
a8 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/xgboost.csv')
a9 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub11_ensemble_imp1.csv')
a10 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub12_catb.csv')
a11 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub3_lgb_ensemble1_01022020.csv')
a12 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub4_ensemble1_01022020.csv')
a13 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub2_lgb_01022020.csv')

In [ ]:
sub = pd.DataFrame()
sub['encounter_id'] = a1['encounter_id']

In [ ]:
pred_ens = (a1['hospital_death'] + a2['hospital_death'] + a3['hospital_death'] + 
            a4['hospital_death'] + a5['hospital_death'] + a6['hospital_death'] +
            a7['hospital_death'] + a8['hospital_death'] + a9['hospital_death'] +
            a10['hospital_death'] + a11['hospital_death'] + a12['hospital_death'] +
            a13['hospital_death']
           )/13.0
            

In [ ]:
sub['hospital_death'] = pred_ens

In [ ]:
sub.to_csv('/Users/s0c02nj/Desktop/WiDS/sub3_ensemble_01022020.csv',index = False)

In [ ]:
#train_x['encounter_id'].unique().shape


In [ ]:
#last let us make the submission,note that you have to make the pred to be int!
pred = model.predict_proba(test_x)
preds= pred[:,1]

In [ ]:
sub_data

In [ ]:
data_sub = x_test[['encounter_id']]
data_sub['hospital_death'] = preds

In [ ]:
x_test.head()

In [ ]:
ls

In [ ]:
data_sub.to_csv('/Users/s0c02nj/Desktop/WiDS/sub2.csv',index = False)

In [ ]:
data_sub.shape